In [16]:
import cv2
import numpy as np
import pytesseract
import easyocr
from matplotlib import pyplot as plt

In [17]:
def display(img):
    cv2.imshow("img",img)
    cv2.waitKey(0)

In [18]:
img=cv2.imread("passport/4.PNG")
max_h,max_w=img.shape[:2]

In [19]:
def display(img):
    cv2.imshow("img",img)
    cv2.waitKey(0)

In [20]:
display(img)

In [21]:
def rotate(image,skew_angle):
    a=0
    if (skew_angle > 45) :
        a=-(90-skew_angle)
    else:
        a=skew_angle
    r,c=image.shape[:2]
    M=cv2.getRotationMatrix2D(((c-1)/2.0,(r-1)/2.0),a,1)  
    return cv2.warpAffine(image,M,(c,r))

In [22]:
def contourize(image):
    img=image.copy()
    inverted=cv2.bitwise_not(img)
    gray=cv2.cvtColor(inverted,cv2.COLOR_BGR2GRAY)
    blur=cv2.GaussianBlur(gray,(5,5),0)
    t=0
    while t < 256:
        thresh=cv2.threshold(blur,t,255,cv2.THRESH_BINARY)[1]
        temp_dilate=cv2.dilate(thresh,cv2.getStructuringElement(cv2.MORPH_RECT,(5,1)),iterations=2)
        temp_contours=cv2.findContours(temp_dilate,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[0]
        temp_mrz=[]
        skew_angles=[]
        for c in temp_contours:
            x,y,w,h=cv2.boundingRect(c)
            if w>=0.5*max_w and h<0.2*max_h and y>0.3*max_h:
                temp_mrz.append(c)
                skew_angles.append(cv2.minAreaRect(c)[-1])
                if len(temp_mrz)==2:
                    return temp_mrz,sum(skew_angles)/len(skew_angles),temp_dilate
        t+=10
    print("mrz is not found")
    return [],0,None           
                

In [23]:
reader=easyocr.Reader(['en'])

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [24]:
def MRZ(image):
    img=image.copy()
    mrz,skew_angle,dilate=contourize(img)
    if mrz==[]:
        print("error")
        return
    box_array=[]
    for c in mrz:
        minarearect=cv2.minAreaRect(c)
        box=cv2.boxPoints(minarearect)
        box32=np.int32(box)
        box_array.append(box32)
    display(cv2.drawContours(img.copy(),box_array,-1,(0,255,0),2))
    temp_img=rotate(img,skew_angle)
    temp_dil=rotate(dilate,skew_angle)
    contours=cv2.findContours(temp_dil,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[0]
    contours=sorted(contours,key= (lambda x: cv2.boundingRect(x)[1]))
    for c in contours:
        (x,y,w,h)= cv2.boundingRect(c)
        if w>0.5*max_w and h<0.2*max_h and y>0.3*max_h:
            margin=5
            x=x-margin
            w=w+2*margin
            y=y-margin          
            h=h+2*margin
            ocr_result=reader.readtext(temp_img[y:y+h,x:x+w],detail=0)
            print(ocr_result)
            display(temp_img[y:y+h,x:x+w])  
                   

In [25]:
MRZ(img)

['P<FIMMAMNARI<<VILLE<VE[KKO<petteri']
['Cp2z485398FIn820826042908133260882<197r<<<6']


In [26]:
def crop_pass(image):
    img=image.copy()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    t=255
    ID=[]
    while t>=0:
        temp_thresh=cv2.threshold(gray,t,255,cv2.THRESH_BINARY)[1]
        temp_dilate=cv2.dilate(temp_thresh,cv2.getStructuringElement(cv2.MORPH_RECT,(6,1)),iterations=5)
        cnts=cv2.findContours(temp_dilate,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[0]
        flag=False
        for c in cnts:
            area=cv2.contourArea(c)
            if area>0.35*max_h*max_w:
                ID.append(c)
                flag=True
        if (flag):
            break
        t-=10
    display(cv2.drawContours(img.copy(),ID,-1,(0,255,0),2))    
    for c in ID:
        (x,y,w,h)=cv2.boundingRect(c)
        display(img[y:y+h,x:x+w])
        return img[y:y+h,x:x+w]       

In [27]:
crop_pass(img)

array([[[ 89, 115, 162],
        [ 93, 119, 166],
        [ 92, 116, 164],
        ...,
        [127, 127, 127],
        [127, 127, 127],
        [127, 127, 127]],

       [[ 90, 116, 163],
        [ 90, 116, 163],
        [ 90, 114, 161],
        ...,
        [127, 127, 127],
        [127, 127, 127],
        [127, 127, 127]],

       [[ 95, 121, 168],
        [ 96, 122, 169],
        [ 93, 117, 165],
        ...,
        [127, 127, 127],
        [127, 127, 127],
        [127, 127, 127]],

       ...,

       [[115, 124, 134],
        [118, 127, 137],
        [120, 129, 139],
        ...,
        [127, 127, 127],
        [127, 127, 127],
        [127, 127, 127]],

       [[118, 127, 137],
        [120, 129, 139],
        [120, 129, 139],
        ...,
        [127, 127, 127],
        [127, 127, 127],
        [127, 127, 127]],

       [[122, 131, 141],
        [123, 132, 142],
        [123, 132, 142],
        ...,
        [127, 127, 127],
        [127, 127, 127],
        [127, 127, 127]]

In [28]:
def contourize(image):
    W  , H , _ = image.shape
    img = image.copy()
    inverted = cv2.bitwise_not(img)
    gray = cv2.cvtColor(inverted , cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray , (5 , 5) , 0)
    t = 0
    while t < 256:
        bw = cv2.threshold(blur , t , 255 , cv2.THRESH_BINARY)[1]
        temp_dilate = cv2.dilate(bw , cv2.getStructuringElement(cv2.MORPH_RECT ,(5 , 1)) , iterations = 2)
        display(temp_dilate)
        temp_contours = cv2.findContours(temp_dilate , cv2.RETR_TREE , cv2.CHAIN_APPROX_NONE)[0]
        temp_mrz= [] 
        skew_angles =[]
        # display(temp_dilate)
        for c in temp_contours:
            (x ,y , w ,h) = cv2.boundingRect(c)
            if w>=0.5*W and h<0.2*H and y>0.3*H:
                temp_mrz.append(c)
                skew_angles.append(cv2.minAreaRect(c)[-1])
        if(len(temp_mrz) == 2):
            # display(cv2.drawContours(img.copy() , mrz , -1 , (0 , 255 , 0) , 2))
            return temp_mrz , sum(skew_angles)/len(skew_angles) , temp_dilate
        t+=10
    print("Unable to detect MRZ")
    return [] , 0 , None

In [29]:
def OCR(image):
    img=image.copy()
    cropped=crop_pass(img)
    mrz,skew_angle,dilate=contourize(cropped)
    skew_angles=[]
    for c in mrz:
        rotatedrect=cv2.minAreaRect(c)
        skew_angles.append(rotatedrect[-1])
    avg_skew=sum(skew_angles)/len(skew_angles)
    deskewed=rotate(cropped,avg_skew)
    mrz,skew_angle,dilate=contourize(deskewed)
    mrz=sorted(mrz,key= lambda x: cv2.boundingRect(x)[1])
    for c in mrz:
        (x,y,w,h)=cv2.boundingRect(c)
        margin=5
        x=x-margin
        w=w+2*margin
        y=y-margin
        h=h+2*margin
        print(reader.readtext(deskewed[y:y+h,x:x+w],detail=0))
        display(deskewed[y:y+h,x:x+w])

In [30]:
OCR(img)

[]
[]
